In [ ]:
!pip install deweydatapy@git+https://github.com/Dewey-Data/deweydatapy
!pip install duckdb

In [1]:
import deweydatapy as ddp

APIKEY = "akv1_QM2Gd4s-bAm-PHHkatlokaVxNzkMISAUXcy"

PPS = {
    'person': "https://api.deweydata.io/api/v1/external/data/fldr_t8vhuaw774eoa4tbj", #782M
    'certification': "https://api.deweydata.io/api/v1/external/data/fldr_dhhd387qjix63dp6g", #277M 
    'degree': "https://api.deweydata.io/api/v1/external/data/fldr_zpjayejai7zf9m7hv", #416M
    'education': "https://api.deweydata.io/api/v1/external/data/fldr_yooaittih4rk7hccp", #672M 
    'experience': "https://api.deweydata.io/api/v1/external/data/fldr_rby46swjrtdkg9z8u", #1.6B 
    'experience_level': "https://api.deweydata.io/api/v1/external/data/fldr_exphf7msjs7z76qof", #420M 
    'interest': "https://api.deweydata.io/api/v1/external/data/fldr_yrcjzcttuxjyduvhh", #308M
    'job_level': "https://api.deweydata.io/api/v1/external/data/fldr_bnux8heehjga7bec8", #132M
    'major': "https://api.deweydata.io/api/v1/external/data/fldr_mia6kqyahgcxhqbxi", #492M
    'minor': "https://api.deweydata.io/api/v1/external/data/fldr_xkbx8rpk8pn7vccjn", #2.1M
    'skill': "https://api.deweydata.io/api/v1/external/data/fldr_rcnddwegyg7aa7r77" #2.4B 
}

RELEVANT_COLUMNS = {
    'person': ['BIRTH_YEAR', 'INDUSTRY', 'JOB_COMPANY_INDUSTRY', 'JOB_LAST_CHANGED', 'JOB_START_DATE',
               'JOB_TITLE', 'JOB_TITLE_CLASS', 'JOB_TITLE_ROLE', 'JOB_TITLE_SUB_ROLE', 'PERSON_ID', 'SEX'],
    'certification': ['NAME', 'PERSON_ID', 'ORGANIZATION', 'START_DATE', 'END_DATE'],
    'degree': ['EDUCATION_ID', 'DEGREE'], # DONE
    'education': ['ID', 'SCHOOL_TYPE', 'GPA', 'START_DATE', 'SCHOOL_NAME', 'END_DATE', 'PERSON_ID'],
    'experience': ['ID', 'COMPANY_NAME', 'END_DATE', 'TITLE_SUB_ROLE', 'TITLE_NAME', 'START_DATE',
                  'COMPANY_INDUSTRY', 'TITLE_ROLE', 'PERSON_ID'],
    'experience_level': ['EXPERIENCE_ID', 'LEVEL'], # DONE
    'interest': ['PERSON_ID', 'INTEREST'],
    'job_level': ['PERSON_ID', 'LEVEL'], # DONE
    'major': ['EDUCATION_ID', 'MAJOR'], # DONE
    'minor': ['EDUCATION_ID', 'MINOR'], # DONE
    'skill': ['PERSON_ID', 'SKILL'] # DONE
}

In [1]:
import duckdb

con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")

In [3]:
person_urls = ddp.get_file_list(APIKEY, PPS['person']).link.to_list()
certification_urls = ddp.get_file_list(APIKEY, PPS['certification']).link.to_list()
degree_urls = ddp.get_file_list(APIKEY, PPS['degree']).link.to_list()
education_urls = ddp.get_file_list(APIKEY, PPS['education']).link.to_list()
experience_urls = ddp.get_file_list(APIKEY, PPS['experience']).link.to_list()
experience_level_urls = ddp.get_file_list(APIKEY, PPS['experience_level']).link.to_list()
interest_urls = ddp.get_file_list(APIKEY, PPS['interest']).link.to_list()
job_level_urls = ddp.get_file_list(APIKEY, PPS['job_level']).link.to_list()
major_urls = ddp.get_file_list(APIKEY, PPS['major']).link.to_list()
minor_urls = ddp.get_file_list(APIKEY, PPS['minor']).link.to_list()
skill_urls = ddp.get_file_list(APIKEY, PPS['skill']).link.to_list()

Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 10
Total number of files: 466
Total files size (MB): 100,305.29
Average single file size (MB): 215.07
Date partition column: None
Expires at: 2025-10-16T06:11:04.374114+00:00
-----------------------------------------------------------------
Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 4
Total number of files: 191
Total files size (MB): 7,071.42
Average single file size (MB): 37.0
Date partition column: None
Expires at: 2025-10-16T06:11:08.160659+00:00
-----------------------------------------------------------------
Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 2
Total number of files: 68
Total files size (MB): 1,846.17
Average single file size (MB): 26.96
Date partition column: None
Expir

In [ ]:
dewey_data_urls = [
    person_urls, certification_urls, degree_urls, education_urls, experience_urls,
    experience_level_urls, interest_urls, job_level_urls, major_urls, minor_urls, skill_urls
]

Download all datasets

In [ ]:
import time

for url, dataset_name in zip(dewey_data_urls, RELEVANT_COLUMNS.keys()):
    columns = RELEVANT_COLUMNS[dataset_name]

    start = time.time()
    
    con.execute(f"""
        SET temp_directory = 'tmp';
        SET preserve_insertion_order = false;
        SET memory_limit='120GB';

        COPY (
            SELECT
                {', '.join(columns)}
            FROM read_csv({url}, compression='gzip')
        ) TO 'dewey_data/{dataset_name}.parquet' (
            FORMAT PARQUET
        );
    """)

    print(f'Finished {dataset_name} in {(time.time() - start) / 60}')

Join lower level datasets into higher level datasets as "MERGED" datasets

This must be done for
* experience_level -> experience
* major, minor, degree -> education
* job_level, skill, certification -> person [not needed atm, to be merged at the end]

NOTE: incidentally some of the code for generating these datasets are missing
* experience_merged is missing

In [ ]:
import time

start = time.time()

con.execute("""
    SET temp_directory = 'tmp';
    SET preserve_insertion_order = false;
    SET memory_limit='120GB';
    
    COPY (
        SELECT
            e.PERSON_ID AS PERSON_ID,
            e.SCHOOL_TYPE AS SCHOOL_TYPE,
            e.GPA AS GPA,
            e.START_DATE AS EDUCATION_START_DATE,
            e.SCHOOL_NAME AS SCHOOL_NAME,
            e.END_DATE AS EDUCATION_END_DATE,
            d.DEGREE AS DEGREE,
            ma.MAJOR AS MAJOR,
            mi.MINOR AS MINOR
        FROM 'dewey_data/education.parquet' e
            LEFT JOIN 'dewey_processed/degree.parquet' d ON e.ID = d.EDUCATION_ID
            LEFT JOIN 'dewey_processed/major.parquet' ma ON e.ID = ma.EDUCATION_ID
            LEFT JOIN 'dewey_processed/minor.parquet' mi ON e.ID = mi.EDUCATION_ID
    )
    TO 'dewey_merged/education_merged.parquet' (
        FORMAT PARQUET
    );
""")

print((time.time() - start) / 60)

education_merged and experience_merged must then be processed to join by person_id with the person dataset

This means that, in the scenario that a person has multiple education and/or experiences, the dataset must be reorganized by person_id
* ie., for education_merged_processed: person_id, [degree_1, degree_2], [major_1, major_2], ...(other education columns)
* same for experience

NOTE: incidentally code for generating education_merged_processed and experience_merged_processed are missing

In [2]:
#

Merging everything into a single dataset
* a for-loop is required as loading all datasets takes too much memory; done in iterations by id instead

In [ ]:
import duckdb, os, time

os.makedirs("test", exist_ok=True)
con = duckdb.connect()

con.execute("""
    SET temp_directory = 'tmp';
    SET preserve_insertion_order = false;
    SET memory_limit = '120GB';
""")

start = time.time()
NUM_BUCKETS = 20

for i in range(NUM_BUCKETS):
    print(f"Processing bucket {i+1}/{NUM_BUCKETS}...")
    con.execute(f"""
        COPY (
            SELECT
                p.PERSON_ID AS PERSON_ID,
                p.BIRTH_YEAR AS BIRTH_YEAR,
                p.INDUSTRY AS INDUSTRY,
                p.JOB_COMPANY_INDUSTRY AS JOB_COMPANY_INDUSTRY,
                p.JOB_LAST_CHANGED AS JOB_LAST_CHANGED,
                p.JOB_START_DATE AS JOB_START_DATE,
                p.JOB_TITLE AS JOB_TITLE,
                p.JOB_TITLE_CLASS AS JOB_TITLE_CLASS,
                p.JOB_TITLE_ROLE AS JOB_TITLE_ROLE,
                p.JOB_TITLE_SUB_ROLE AS JOB_TITLE_SUB_ROLE,
                p.SEX AS SEX,
                jl.LEVEL AS JOB_LEVEL,
                i.INTEREST AS INTEREST,
                s.SKILL AS SKILL,
                c.NAME AS CERTIFICATION_NAME,
                c.ORGANIZATION AS CERTIFICATION_ORGANIZATION,
                c.START_DATE AS CERTIFICATION_START_DATE,
                c.END_DATE AS CERTIFICATION_END_DATE,
                edu.SCHOOL_TYPE AS SCHOOL_TYPE,
                edu.GPA AS GPA,
                edu.EDUCATION_START_DATE AS EDUCATION_START_DATE,
                edu.SCHOOL_NAME AS SCHOOL_NAME,
                edu.EDUCATION_END_DATE AS EDUCATION_END_DATE,
                edu.DEGREE AS DEGREE,
                edu.MAJOR AS MAJOR,
                edu.MINOR AS MINOR,
                exp.EXPERIENCE_COMPANY_NAME AS EXPERIENCE_COMPANY_NAME,
                exp.EXPERIENCE_END_DATE AS EXPERIENCE_END_DATE,
                exp.EXPERIENCE_TITLE_SUB_ROLE AS EXPERIENCE_TITLE_SUB_ROLE,
                exp.EXPERIENCE_TITLE_NAME AS EXPERIENCE_TITLE_NAME,
                exp.EXPERIENCE_START_DATE AS EXPERIENCE_START_DATE,
                exp.EXPERIENCE_COMPANY_INDUSTRY AS EXPERIENCE_COMPANY_INDUSTRY,
                exp.EXPERIENCE_TITLE_ROLE AS EXPERIENCE_TITLE_ROLE,
                exp.EXPERIENCE_LEVEL AS EXPERIENCE_LEVEL
            FROM 'dewey_data/person_us.parquet' p
                LEFT JOIN 'dewey_processed/certification.parquet' c 
                    ON p.PERSON_ID = c.PERSON_ID
                LEFT JOIN 'dewey_processed/job_level.parquet' jl 
                    ON p.PERSON_ID = jl.PERSON_ID
                LEFT JOIN 'dewey_processed/interest.parquet' i 
                    ON p.PERSON_ID = i.PERSON_ID
                LEFT JOIN 'dewey_processed/skill.parquet' s 
                    ON p.PERSON_ID = s.PERSON_ID
                LEFT JOIN 'dewey_merged/experience_merged_processed.parquet' exp 
                    ON p.PERSON_ID = exp.PERSON_ID
                LEFT JOIN 'dewey_merged/education_merged_processed.parquet' edu 
                    ON p.PERSON_ID = edu.PERSON_ID
            WHERE MOD(HASH(p.PERSON_ID), {NUM_BUCKETS}) = {i}
        )
        TO 'test/final_bucket_us_{i}.parquet' (FORMAT PARQUET);
    """)

print("Merging all partial buckets...")
con.execute("""
    COPY (
        SELECT * FROM read_parquet('test/final_bucket_us_*.parquet')
    )
    TO 'dewey_final/final_us.parquet' (FORMAT PARQUET);
""")

print(f"Finished in {(time.time()-start)/60:.2f} minutes")
